In [61]:
from pyspark.sql.functions import col,when
from pyspark.sql import SparkSession
import os


In [5]:

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()


In [19]:
# Here we will preprocess before insterting in our DB
# TODO: See if its better to combine csv and then add, or add 1 by 1

files = [f for f in os.listdir('.') if os.path.isfile(f)]
for f in files:
    filename = f.split(".")
    if filename[1] == "csv" and ("train" in filename[0]):
        print(filename[0])

train-1
train-2
train-3
train-4
train-5
train-6
train-7
train-8


In [119]:
# How to read csv's
df1 = spark.read.option("delimiter", ",").option("header", True).csv('train-2.csv')
df1.count()

993

In [120]:
df1 = df1.na.drop(how = "any",subset=["numVotes"])
# Doesnt make sense to build a ratings model without having the number
# of votes available, reduces interpretability as well, so we remove rows without numVotes

df1 = df1.withColumn("startYear", \
       when(col("startYear")=="\\N" ,col("endYear")) \
          .otherwise(col("startYear"))) 
# This is a mistake, so when there is no startYear available we replace it with endYear

df1 = df1.withColumn("originalTitle", \
       when(col("originalTitle")==col("primaryTitle") ,"null") \
          .otherwise(col("originalTitle"))) 
# When there is a similar original and primary title, remove the original title
# since original title is reserved for non-English films and it adds noise to our data

df1 = df1.drop("endYear")
# There is no startYear and endYear in movies, so delete endYear

df1 =df1.withColumnRenamed("startYear","year")
# Change name of startYear column to year for obvious reasons


In [94]:
df1.show()